In [1]:
pip install azure-cognitiveservices-vision-customvision==2.0.0

     |████████████████████████████████| 54 kB 3.5 MB/s 
     |████████████████████████████████| 85 kB 5.0 MB/s 
     |████████████████████████████████| 45 kB 4.7 MB/s 


In [2]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageFileCreateEntry, Region
from msrest.authentication import ApiKeyCredentials
import os, time, uuid

In [3]:
# Replace with valid values
endpoint = "https://mincustomvisionex.cognitiveservices.azure.com/"
training_key = "training_key"
prediction_key = "prediction_key"
prediction_resource_id = "/subscriptions/80168875-775a-4c14-baa6-2f9817cc98cc/resourceGroups/CognitiveServiceTest/providers/Microsoft.CognitiveServices/accounts/mincustomvisionex"

In [4]:
credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
trainer = CustomVisionTrainingClient(endpoint, credentials)
prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
predictor = CustomVisionPredictionClient(endpoint, prediction_credentials)

In [5]:
publish_iteration_name = "classifyModel"

credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
trainer = CustomVisionTrainingClient(endpoint, credentials)

# Create a new project
print ("Creating project...")
project_name = uuid.uuid4()
project = trainer.create_project(project_name)

Creating project...


In [6]:
# Make two tags in the new project
hemlock_tag = trainer.create_tag(project.id, "Hemlock")
cherry_tag = trainer.create_tag(project.id, "Japanese Cherry")

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
base_image_location = os.path.join("/content/drive/MyDrive/Colab Notebooks/", "Images")

print("Adding images...")

image_list = []

for image_num in range(1, 11):
  file_name = "hemlock_{}.jpg".format(image_num)
  with open(os.path.join(base_image_location, "Hemlock", file_name),"rb") as image_contents:
    image_list.append(ImageFileCreateEntry(name=file_name, contents=image_contents.read(), tag_ids=[hemlock_tag.id]))

for image_num in range(1, 11):
  file_name = "japanese_cherry_{}.jpg".format(image_num)
  with open(os.path.join(base_image_location, "Japanese_Cherry", file_name), "rb") as image_contents:
    image_list.append(ImageFileCreateEntry(name=file_name, contents=image_contents.read(), tag_ids=[cherry_tag.id]))

upload_result = trainer.create_images_from_files(project.id, ImageFileCreateBatch(images=image_list))
if not upload_result.is_batch_successful:
  print("Image batch upload filed")
  for image in upload_result.images:
    print("Image status: ", image.status)
  exit(-1)

Adding images...


TypeError: ignored

In [ ]:
print("Training...")
iteration = trainer.train_project(project.id)
while (iteration.status != "Completed"):
  iteration = trainer.get_iteration(project.id, iteration.id)
  print("Training status: " + iteration.status)
  print("Waiting 10 seconds...")
  time.sleep(5)

In [ ]:
# The iteration is now trained. Publish it to the project endpoint
trainer.publish_iteration(project.id, iteration.id, publish_iteration_name, prediction_resource_id)
print("Done!")

In [ ]:
# Now there is a trained endpoint that can be used to make a prediction
prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
predictor = CustomVisionPredictionClient(endpoint, prediction_credentials)

with open(os.path.join (base_image_location, "Test/test_image.jpg"), "rb") as image_contents:
    results = predictor.classify_image(project.id, publish_iteration_name, image_contents.read())

    # Display the results.
    for prediction in results.predictions:
        print("\t" + prediction.tag_name +
              ": {0:.2f}%".format(prediction.probability * 100))